In [8]:
!pip install pydantic

In [9]:
from pydantic.config import ConfigDict

In [10]:
class MultiSearchQueryResponse(BaseModel):
    products: List[SearchQuery]

    model_config = ConfigDict(
      json_schema_extra={
            "examples": [
                {
                    "products": [
                        {
                            "product_name": "Nike Air Max",
                            "query": "black running shoes",
                        },
                        {
                            "product_name": "Apple iPhone 13",
                            "query": "smartphone with best camera",
                        },
                    ]
                }
            ]
        }
    )

In [14]:
!pip install openai
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.6 MB/s eta 0:00:00


In [15]:
!jupyter notebook restart

[C 18:33:37.435 NotebookApp] No such file or directory: /content/restart


enter your API key for GPT4

In [ ]:
from openai import OpenAI
from fastapi import FastAPI
from pydantic import BaseModel, ConfigDict
from typing import List

client = OpenAI(api_key="YOUR_OPENAI_API_KEY")

app = FastAPI(
    title="Ecommerce Vision API",
    description="""A FastAPI application to extract products
        from images and describe them as an array of queries""",
    version="0.1.0",
)


class ImageRequest(BaseModel):
    url: str
    temperature: float = 0.0
    max_tokens: int = 1800

    model_config = ConfigDict(
      json_schema_extra={
            "examples": [
                {
                    "url": "https://mensfashionpostingcom.files.wordpress.com/2020/03/fbe79-img_5052.jpg?w=768",
                    "temperature": 0.0,
                    "max_tokens": 1800,
                }
            ]
        }
    )

class MultiSearchQuery(BaseModel):
    product_name: str
    query: str

class MultiSearchQueryResponse(BaseModel):
    products: List[MultiSearchQuery]

    @classmethod
    def model_json_schema(cls):
        return {
            "type": "object",
            "properties": {
                "products": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "product_name": {"type": "string"},
                            "query": {"type": "string"},
                        },
                        "required": ["product_name", "query"],
                    },
                }
            },
            "required": ["products"],
        }

    @classmethod
    def model_validate_json(cls, json_data):
        return cls.model_validate(json.loads(json_data))

@app.post("/api/extract_products", response_model=MultiSearchQueryResponse)
async def extract_products(image_request: ImageRequest) -> MultiSearchQueryResponse:
    completion = await client.chat.completions.create(
        model="gpt-4-vision-preview",
        max_tokens=request.max_tokens,
        temperature=request.temperature,
        stop=["```"],
        messages=[
            {
                "role": "system",
                "content": f"""
                You are an expert system designed to extract products from images for
                an ecommerce application. Please provide the product name and a
                descriptive query to search for the product. Accurately identify every
                product in an image and provide a descriptive query to search for the
                product. You just return a correctly formatted JSON object with the
                product name and query for each product in the image and follows the
                schema below:

                JSON Schema:
                {MultiSearchQueryResponse.model_json_schema()}""",
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """Extract the products from the image,
                        and describe them in a query in JSON format""",
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": request.url},
                    },
                ],
            },
            {
                "role": "assistant",
                "content": "```json",
            },
        ],
    )
    return MultiSearchQueryResponse.model_validate_json(completion.choices[0].message.content)